In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
_INPUT_DATA_PATH = ['./FC_2021-03.csv',
                    './FC_2021-04.csv',
                    './FC_2021-05.csv',
                    './FC_2021-06.csv']


In [3]:
df = pd.DataFrame()
for data_path in _INPUT_DATA_PATH:
    temp_df = pd.read_csv(data_path, low_memory=False, dtype={
                     'BKG_DATE': str, 'BKG_TIME': str, 'POST_ZONE': str
                 })
    df = pd.concat([df, temp_df], ignore_index=True)

    print(data_path, "\t", temp_df.shape)

print(df.info())

./FC_2021-03.csv 	 (309352, 32)
./FC_2021-04.csv 	 (349588, 32)
./FC_2021-05.csv 	 (450522, 32)
./FC_2021-06.csv 	 (676912, 32)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1786374 entries, 0 to 1786373
Data columns (total 32 columns):
 #   Column              Dtype  
---  ------              -----  
 0   CORP_ID             object 
 1   REF_ORD_NO          object 
 2   BKG_NO              object 
 3   BKG_TYP             int64  
 4   BKG_DATE            object 
 5   BKG_TIME            object 
 6   SHPR_CD             int64  
 7   INV_AMT             float64
 8   ITEM_SEQ            int64  
 9   ITEM_CD             object 
 10  BRAND_NM            object 
 11  ITEM_QTY            int64  
 12  ITEM_AMT            float64
 13  IF_YN               object 
 14  ORDER_CRT_DATETIME  float64
 15  DLV_DV              float64
 16  REF_ITEM_SEQ        object 
 17  ORDER_IDX           object 
 18  ORDER_YN            object 
 19  DLVPREARRBRANCD     object 
 20  DLVPREARREMPNICKNM  object 


In [4]:
_cols = {'CORP_ID', 'BKG_NO', 'INS_ID', 'INS_DATE', 'INS_TIME', 'BRAND_NM',
         'ORDER_IDX', 'ORDER_YN', 'SPLIT_EXEC_YN', 'DLVCLSFCD', 'DLVSUBCLSFCD', 
         'DLVPREARRBRANCD', 'DLVPREARREMPNICKNM', 'DLV_DV', 'REF_ITEM_SEQ', 'IF_YN',
         'ORDER_CRT_DATETIME', 'SHPR_ADDR_1', 'SHPR_ADDR_2', 'CNEE_ADDR_1', 'CNEE_ADDR_2','BKG_TYP','ITEM_SEQ','REF_ORD_NO','POST_ZONE','ITEM_QTY','BKG_TIME'}

fc = df.loc[df['CORP_ID'] == 'KX007'].drop(columns=_cols)
fc.head(5)

,BKG_DATE,SHPR_CD,INV_AMT,ITEM_CD,ITEM_AMT
0,20210303,90001302,22400.0,134265392,14900.0
1,20210303,90001441,20900.0,4610261043_2,20900.0
2,20210303,90001441,37900.0,4936599538_1,37900.0
3,20210303,90001542,69800.0,3858391637_1,69800.0
4,20210303,90001341,73700.0,4631307460,39800.0


In [5]:
fc['BKG_DATE'] = pd.to_datetime(fc['BKG_DATE'])
#fc['BKG_TIME'] = pd.to_datetime(fc['BKG_TIME'], format="%H%M%S").dt.time

In [6]:
# 1441 고객사

fc_1302 = fc[fc['SHPR_CD'] == 90001541]
fc_1302.drop(columns='SHPR_CD')

,BKG_DATE,INV_AMT,ITEM_CD,ITEM_AMT
15,2021-03-03,79800.0,5329176574,39900.0
20,2021-03-03,31990.0,4929464978_1,31990.0
53,2021-03-01,34700.0,4802177320_8,15800.0
54,2021-03-01,34700.0,4802177320_2,7900.0
55,2021-03-01,34700.0,4824061918_29,11000.0
...,...,...,...,...
1786017,2021-06-29,32700.0,4861386623_6,10900.0
1786018,2021-06-29,32700.0,4861386623_7,21800.0
1786033,2021-06-29,63980.0,4929464978_2,31990.0
1786034,2021-06-29,63980.0,4929464978_1,31990.0


In [7]:
# 1
fc_1302_inv = fc_1302[['BKG_DATE','INV_AMT']]
fc_1302_inv = fc_1302_inv.groupby(['BKG_DATE']).sum()
fc_1302_inv

,INV_AMT
BKG_DATE,
2021-03-01,42833300.0
2021-03-02,71994810.0
2021-03-03,69231210.0
2021-03-04,57583280.0
2021-03-05,46071200.0
...,...
2021-06-26,33109980.0
2021-06-27,50137770.0
2021-06-28,95021220.0


In [8]:
fc_1302_week = fc_1302_inv.resample(rule='1W')
fc_1302_week = fc_1302_week.sum()
fc_1302_week

,INV_AMT
BKG_DATE,
2021-03-07,340706230.0
2021-03-14,351188290.0
2021-03-21,421517610.0
2021-03-28,513069500.0
2021-04-04,494157190.0
2021-04-11,493461210.0
2021-04-18,930317220.0
2021-04-25,539641710.0
2021-05-02,547639130.0


In [9]:
fc_1302_inv = pd.concat([fc_1302_inv,fc_1302_week],axis=1)
fc_1302_inv = fc_1302_inv.fillna(method = 'bfill')
fc_1302_inv = fc_1302_inv[0:-1][:]
fc_1302_inv.columns = ['INV_AMT','INV_AMT_WEEK']                     

In [10]:
fc_1302_inv['INV_Q1'] = fc_1302_inv['INV_AMT_WEEK']*0.25
fc_1302_inv['INV_Q2'] = fc_1302_inv['INV_AMT_WEEK']*0.5
fc_1302_inv['INV_Q3'] = fc_1302_inv['INV_AMT_WEEK']*0.75
fc_1302_inv

,INV_AMT,INV_AMT_WEEK,INV_Q1,INV_Q2,INV_Q3
BKG_DATE,,,,,
2021-03-01,42833300.0,340706230.0,85176557.5,170353115.0,255529672.5
2021-03-02,71994810.0,340706230.0,85176557.5,170353115.0,255529672.5
2021-03-03,69231210.0,340706230.0,85176557.5,170353115.0,255529672.5
2021-03-04,57583280.0,340706230.0,85176557.5,170353115.0,255529672.5
2021-03-05,46071200.0,340706230.0,85176557.5,170353115.0,255529672.5
...,...,...,...,...,...
2021-06-26,33109980.0,440726560.0,110181640.0,220363280.0,330544920.0
2021-06-27,50137770.0,440726560.0,110181640.0,220363280.0,330544920.0
2021-06-28,95021220.0,246152430.0,61538107.5,123076215.0,184614322.5


In [11]:
fc_1302_inv = fc_1302_inv.drop(columns='INV_AMT_WEEK')
fc_1302_inv

,INV_AMT,INV_Q1,INV_Q2,INV_Q3
BKG_DATE,,,,
2021-03-01,42833300.0,85176557.5,170353115.0,255529672.5
2021-03-02,71994810.0,85176557.5,170353115.0,255529672.5
2021-03-03,69231210.0,85176557.5,170353115.0,255529672.5
2021-03-04,57583280.0,85176557.5,170353115.0,255529672.5
2021-03-05,46071200.0,85176557.5,170353115.0,255529672.5
...,...,...,...,...
2021-06-26,33109980.0,110181640.0,220363280.0,330544920.0
2021-06-27,50137770.0,110181640.0,220363280.0,330544920.0
2021-06-28,95021220.0,61538107.5,123076215.0,184614322.5


In [12]:
for i, row in fc_1302_inv.iterrows():
    INV_AMT = row['INV_AMT']
    q1 = row['INV_Q1']
    q2 = row['INV_Q2']
    q3 = row['INV_Q3']
    
    if INV_AMT < q1:
        row['INV_AMT'] = 1
    elif q1 <= INV_AMT < q2:
        row['INV_AMT'] = 2
    elif q2 <= INV_AMT < q3:
        row['INV_AMT'] = 3
    elif q3 <= INV_AMT:
        row['INV_AMT'] = 4
    else:
        row['INV_AMT'] = 0 # 오류확인
        
    fc_1302_inv.loc[i] = row
    
print(fc_1302_inv)

            INV_AMT       INV_Q1       INV_Q2       INV_Q3
BKG_DATE                                                  
2021-03-01      1.0   85176557.5  170353115.0  255529672.5
2021-03-02      1.0   85176557.5  170353115.0  255529672.5
2021-03-03      1.0   85176557.5  170353115.0  255529672.5
2021-03-04      1.0   85176557.5  170353115.0  255529672.5
2021-03-05      1.0   85176557.5  170353115.0  255529672.5
...             ...          ...          ...          ...
2021-06-26      1.0  110181640.0  220363280.0  330544920.0
2021-06-27      1.0  110181640.0  220363280.0  330544920.0
2021-06-28      2.0   61538107.5  123076215.0  184614322.5
2021-06-29      2.0   61538107.5  123076215.0  184614322.5
2021-06-30      2.0   61538107.5  123076215.0  184614322.5

[122 rows x 4 columns]


In [13]:
fc_1302_inv['INV_AMT'].value_counts()

1.0    113
2.0      8
3.0      1
Name: INV_AMT, dtype: int64

In [14]:
fc_1302_inv = fc_1302_inv.drop(columns=['INV_Q1','INV_Q2','INV_Q3'])
fc_1302_inv = fc_1302_inv.reset_index()
fc_1302_inv

,BKG_DATE,INV_AMT
0,2021-03-01,1.0
1,2021-03-02,1.0
2,2021-03-03,1.0
3,2021-03-04,1.0
4,2021-03-05,1.0
...,...,...
117,2021-06-26,1.0
118,2021-06-27,1.0
119,2021-06-28,2.0
120,2021-06-29,2.0


In [15]:
## item amt 

fc_1302_amt = fc_1302[['BKG_DATE','ITEM_AMT']]
fc_1302_amt = fc_1302_amt.groupby(['BKG_DATE']).sum()

fc_1302_week = fc_1302_amt.resample(rule='1W')
fc_1302_week = fc_1302_week.sum()


In [16]:
fc_1302_amt = pd.concat([fc_1302_amt,fc_1302_week],axis=1)
fc_1302_amt = fc_1302_amt.fillna(method = 'bfill')
fc_1302_amt = fc_1302_amt[0:-1][:]
fc_1302_amt.columns = ['ITEM_AMT','ITEM_AMT_WEEK'] 
fc_1302_amt

,ITEM_AMT,ITEM_AMT_WEEK
BKG_DATE,,
2021-03-01,16684520.0,123619250.0
2021-03-02,25119070.0,123619250.0
2021-03-03,22506460.0,123619250.0
2021-03-04,20581270.0,123619250.0
2021-03-05,15991320.0,123619250.0
...,...,...
2021-06-26,15435550.0,197844680.0
2021-06-27,22107370.0,197844680.0
2021-06-28,43567930.0,108103790.0


In [17]:
fc_1302_amt['ITEM_Q1'] = fc_1302_amt['ITEM_AMT_WEEK']*0.25
fc_1302_amt['ITEM_Q2'] = fc_1302_amt['ITEM_AMT_WEEK']*0.5
fc_1302_amt['ITEM_Q3'] = fc_1302_amt['ITEM_AMT_WEEK']*0.75
fc_1302_amt

,ITEM_AMT,ITEM_AMT_WEEK,ITEM_Q1,ITEM_Q2,ITEM_Q3
BKG_DATE,,,,,
2021-03-01,16684520.0,123619250.0,30904812.5,61809625.0,92714437.5
2021-03-02,25119070.0,123619250.0,30904812.5,61809625.0,92714437.5
2021-03-03,22506460.0,123619250.0,30904812.5,61809625.0,92714437.5
2021-03-04,20581270.0,123619250.0,30904812.5,61809625.0,92714437.5
2021-03-05,15991320.0,123619250.0,30904812.5,61809625.0,92714437.5
...,...,...,...,...,...
2021-06-26,15435550.0,197844680.0,49461170.0,98922340.0,148383510.0
2021-06-27,22107370.0,197844680.0,49461170.0,98922340.0,148383510.0
2021-06-28,43567930.0,108103790.0,27025947.5,54051895.0,81077842.5


In [18]:
fc_1302_amt = fc_1302_amt.drop(columns='ITEM_AMT_WEEK')
fc_1302_amt

,ITEM_AMT,ITEM_Q1,ITEM_Q2,ITEM_Q3
BKG_DATE,,,,
2021-03-01,16684520.0,30904812.5,61809625.0,92714437.5
2021-03-02,25119070.0,30904812.5,61809625.0,92714437.5
2021-03-03,22506460.0,30904812.5,61809625.0,92714437.5
2021-03-04,20581270.0,30904812.5,61809625.0,92714437.5
2021-03-05,15991320.0,30904812.5,61809625.0,92714437.5
...,...,...,...,...
2021-06-26,15435550.0,49461170.0,98922340.0,148383510.0
2021-06-27,22107370.0,49461170.0,98922340.0,148383510.0
2021-06-28,43567930.0,27025947.5,54051895.0,81077842.5


In [19]:
for i, row in fc_1302_amt.iterrows():
    ITEM_AMT = row['ITEM_AMT']
    q1 = row['ITEM_Q1']
    q2 = row['ITEM_Q2']
    q3 = row['ITEM_Q3']
    
    if ITEM_AMT < q1:
        row['ITEM_AMT'] = 1
    elif q1 <= ITEM_AMT < q2:
        row['ITEM_AMT'] = 2
    elif q2 <= ITEM_AMT < q3:
        row['ITEM_AMT'] = 3
    elif q3 <= ITEM_AMT:
        row['ITEM_AMT'] = 4
    else:
        row['ITEM_AMT'] = 0 # 오류확인
        
    fc_1302_amt.loc[i] = row
    
print(fc_1302_amt)

            ITEM_AMT     ITEM_Q1     ITEM_Q2      ITEM_Q3
BKG_DATE                                                 
2021-03-01       1.0  30904812.5  61809625.0   92714437.5
2021-03-02       1.0  30904812.5  61809625.0   92714437.5
2021-03-03       1.0  30904812.5  61809625.0   92714437.5
2021-03-04       1.0  30904812.5  61809625.0   92714437.5
2021-03-05       1.0  30904812.5  61809625.0   92714437.5
...              ...         ...         ...          ...
2021-06-26       1.0  49461170.0  98922340.0  148383510.0
2021-06-27       1.0  49461170.0  98922340.0  148383510.0
2021-06-28       2.0  27025947.5  54051895.0   81077842.5
2021-06-29       2.0  27025947.5  54051895.0   81077842.5
2021-06-30       2.0  27025947.5  54051895.0   81077842.5

[122 rows x 4 columns]


In [20]:
fc_1302_amt = fc_1302_amt.drop(columns=['ITEM_Q1','ITEM_Q2','ITEM_Q3'])
fc_1302_amt = fc_1302_amt.reset_index()
fc_1302_amt['ITEM_AMT'].value_counts()

1.0    114
2.0      4
3.0      4
Name: ITEM_AMT, dtype: int64

In [21]:
fc_1302_inv['ITEM_AMT'] = fc_1302_amt['ITEM_AMT']

In [22]:
fc_1302_inv['weekday'] = fc_1302_inv['BKG_DATE'].dt.dayofweek
fc_1302_inv['weekday'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [23]:
n = 0

for i in fc_1302_inv['weekday']:
    if fc_1302_inv['weekday'][n] == 0:
        fc_1302_inv['weekday'][n] = 1
    else:
        fc_1302_inv['weekday'][n] = 0
    n += 1
    if n == 122:
        break

/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17129/1473546988.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['weekday'][n] = 1
/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17129/1473546988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['weekday'][n] = 0


In [24]:
from pytimekr import pytimekr
holiday = pytimekr.holidays(year=2021)

fc_1302_inv['holiday'] = fc_1302_inv['BKG_DATE']

In [25]:
# 휴일 변수 할당

a='2021-03-01'
a= pd.to_datetime(a)
b='2021-05-05'
b= pd.to_datetime(b)
c='2021-05-19'
c= pd.to_datetime(c)
d='2021-06-06'
d= pd.to_datetime(d)

In [26]:
# 휴일 

for i, row in fc_1302_inv.iterrows():
    holiday = row['holiday']

    if holiday == a :
         row['holiday'] = 1
    elif holiday == b :
        row['holiday'] = 1
    elif holiday == c :
        row['holiday'] = 1
    elif holiday == d :
        row['holiday'] = 1        
    else:
         row['holiday'] = 0
   
    fc_1302_inv.loc[i] = row
    
print(fc_1302_inv)

      BKG_DATE  INV_AMT  ITEM_AMT  weekday holiday
0   2021-03-01      1.0       1.0        1       1
1   2021-03-02      1.0       1.0        0       0
2   2021-03-03      1.0       1.0        0       0
3   2021-03-04      1.0       1.0        0       0
4   2021-03-05      1.0       1.0        0       0
..         ...      ...       ...      ...     ...
117 2021-06-26      1.0       1.0        0       0
118 2021-06-27      1.0       1.0        0       0
119 2021-06-28      2.0       2.0        1       0
120 2021-06-29      2.0       2.0        0       0
121 2021-06-30      2.0       2.0        0       0

[122 rows x 5 columns]


In [27]:
# 휴일 하루 전

fc_1302_inv['holiday_diff'] = fc_1302_inv['BKG_DATE']

In [28]:
# 휴일 전 변수 할당

q='2021-02-28'
q= pd.to_datetime(q)
w='2021-05-04'
w= pd.to_datetime(w)
e='2021-05-18'
e= pd.to_datetime(e)
r='2021-06-05'
r= pd.to_datetime(r)

In [29]:
# 휴일 하루 전

fc_1302_inv['holiday_diff'] = fc_1302_inv['BKG_DATE']


# 휴일 전 변수 할당

q='2021-02-28'
q= pd.to_datetime(q)
w='2021-05-04'
w= pd.to_datetime(w)
e='2021-05-18'
e= pd.to_datetime(e)
r='2021-06-05'
r= pd.to_datetime(r)



for i, row in fc_1302_inv.iterrows():
    holiday_diff = row['holiday_diff']

    if holiday_diff == q :
         row['holiday_diff'] = 1
    elif holiday_diff == w :
        row['holiday_diff'] = 1
    elif holiday_diff == e :
        row['holiday_diff'] = 1
    elif holiday_diff == r :
        row['holiday_diff'] = 1        
    else:
         row['holiday_diff'] = 0
   
    fc_1302_inv.loc[i] = row
    
print(fc_1302_inv)

      BKG_DATE  INV_AMT  ITEM_AMT  weekday holiday holiday_diff
0   2021-03-01      1.0       1.0        1       1            0
1   2021-03-02      1.0       1.0        0       0            0
2   2021-03-03      1.0       1.0        0       0            0
3   2021-03-04      1.0       1.0        0       0            0
4   2021-03-05      1.0       1.0        0       0            0
..         ...      ...       ...      ...     ...          ...
117 2021-06-26      1.0       1.0        0       0            0
118 2021-06-27      1.0       1.0        0       0            0
119 2021-06-28      2.0       2.0        1       0            0
120 2021-06-29      2.0       2.0        0       0            0
121 2021-06-30      2.0       2.0        0       0            0

[122 rows x 6 columns]


In [30]:
### ITEM_CD count

cd = fc_1302[['BKG_DATE','ITEM_CD']]
cd = cd.groupby(['BKG_DATE']).count()
cd = cd.reset_index()
fc_1302_inv['ITEM_CD_COUNT'] = cd['ITEM_CD']

In [31]:
diff1 = fc_1302_inv['INV_AMT'][:-1]
fc_1302_inv['INV_AMT_DIFF'] = fc_1302_inv['INV_AMT']
fc_1302_inv['INV_AMT_DIFF'].iloc[1:] = diff1
fc_1302_inv

/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17129/3734826041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['INV_AMT_DIFF'].iloc[1:] = diff1


,BKG_DATE,INV_AMT,ITEM_AMT,weekday,holiday,holiday_diff,ITEM_CD_COUNT,INV_AMT_DIFF
0,2021-03-01,1.0,1.0,1,1,0,1035,1.0
1,2021-03-02,1.0,1.0,0,0,0,1371,1.0
2,2021-03-03,1.0,1.0,0,0,0,1141,1.0
3,2021-03-04,1.0,1.0,0,0,0,1137,1.0
4,2021-03-05,1.0,1.0,0,0,0,811,1.0
...,...,...,...,...,...,...,...,...
117,2021-06-26,1.0,1.0,0,0,0,649,1.0
118,2021-06-27,1.0,1.0,0,0,0,946,1.0
119,2021-06-28,2.0,2.0,1,0,0,1436,1.0
120,2021-06-29,2.0,2.0,0,0,0,1128,2.0


In [32]:
diff2 = fc_1302_inv['ITEM_AMT'][:-1]
fc_1302_inv['ITEM_AMT_DIFF'] = fc_1302_inv['ITEM_AMT']
fc_1302_inv['ITEM_AMT_DIFF'].iloc[1:] = diff2
fc_1302_inv

/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17129/3619347445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['ITEM_AMT_DIFF'].iloc[1:] = diff2


,BKG_DATE,INV_AMT,ITEM_AMT,weekday,holiday,holiday_diff,ITEM_CD_COUNT,INV_AMT_DIFF,ITEM_AMT_DIFF
0,2021-03-01,1.0,1.0,1,1,0,1035,1.0,1.0
1,2021-03-02,1.0,1.0,0,0,0,1371,1.0,1.0
2,2021-03-03,1.0,1.0,0,0,0,1141,1.0,1.0
3,2021-03-04,1.0,1.0,0,0,0,1137,1.0,1.0
4,2021-03-05,1.0,1.0,0,0,0,811,1.0,1.0
...,...,...,...,...,...,...,...,...,...
117,2021-06-26,1.0,1.0,0,0,0,649,1.0,1.0
118,2021-06-27,1.0,1.0,0,0,0,946,1.0,1.0
119,2021-06-28,2.0,2.0,1,0,0,1436,1.0,1.0
120,2021-06-29,2.0,2.0,0,0,0,1128,2.0,2.0


In [33]:
fc_1302_inv = fc_1302_inv[['BKG_DATE','INV_AMT','INV_AMT_DIFF','ITEM_AMT','ITEM_AMT_DIFF','weekday','holiday','holiday_diff','ITEM_CD_COUNT']]
fc_1302_inv

,BKG_DATE,INV_AMT,INV_AMT_DIFF,ITEM_AMT,ITEM_AMT_DIFF,weekday,holiday,holiday_diff,ITEM_CD_COUNT
0,2021-03-01,1.0,1.0,1.0,1.0,1,1,0,1035
1,2021-03-02,1.0,1.0,1.0,1.0,0,0,0,1371
2,2021-03-03,1.0,1.0,1.0,1.0,0,0,0,1141
3,2021-03-04,1.0,1.0,1.0,1.0,0,0,0,1137
4,2021-03-05,1.0,1.0,1.0,1.0,0,0,0,811
...,...,...,...,...,...,...,...,...,...
117,2021-06-26,1.0,1.0,1.0,1.0,0,0,0,649
118,2021-06-27,1.0,1.0,1.0,1.0,0,0,0,946
119,2021-06-28,2.0,1.0,2.0,1.0,1,0,0,1436
120,2021-06-29,2.0,2.0,2.0,2.0,0,0,0,1128


In [34]:
fc_1302_train = fc_1302_inv.iloc[:92]
fc_1302_test = fc_1302_inv.iloc[92:]

In [35]:
fc_1302_train.to_csv('fc_1541_train.csv',index=False)
fc_1302_test.to_csv('fc_1541_test.csv',index=False)